In [1]:
%matplotlib inline

import sys, os
sys.path.append("/home/daniil/repos/pytorch-segmentation-detection/")
sys.path.insert(0, '/home/daniil/repos/pytorch-segmentation-detection/vision/')
sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/build/lib.linux-x86_64-2.7")

# Use second GPU -pytorch-segmentation-detection- change if you want to use a first one
os.environ["CUDA_VISIBLE_DEVICES"] = '1'


from PIL import Image
from matplotlib import pyplot as plt

import torch
import torch.onnx
from torchvision import transforms
from torch.autograd import Variable
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated

import numpy as np

img_path = '/home/daniil/datasets/data/instrument_dataset_1/left_frames/frame000.png'

valid_transform = transforms.Compose(
                [
                    transforms.CenterCrop((512, 512)),
                     transforms.ToTensor(),
                     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                ])

img_not_preprocessed = Image.open(img_path).convert('RGB')#.resize((512, 512))

img = valid_transform(img_not_preprocessed)

img = img.unsqueeze(0)

img = img.cuda()

fcn = resnet_dilated.Resnet18_8s(num_classes=2)
fcn.load_state_dict(torch.load('/home/daniil/models/endovis_2017/resnet_18_8s_best.pth'))
fcn.cuda()
fcn.eval()

# res = fcn(img)

# _, tmp = res.squeeze(0).max(0)
# segmentation = tmp.data.cpu().numpy().squeeze()

# plt.imshow(img_not_preprocessed)
# plt.show()

# plt.imshow(segmentation)
# plt.show()

Resnet18_8s(
  (resnet18_8s): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [2]:
# input and output names can be omitted -- the input will be named '0' in that case
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(122) ]
output_names = [ "output1" ]

torch.onnx.export(fcn, img, "resnet_18_8s_binary_512.onnx", verbose=True, input_names=input_names, output_names=output_names)

graph(%actual_input_1 : Float(1, 3, 512, 512)
      %learned_0 : Float(64, 3, 7, 7)
      %learned_1 : Float(64)
      %learned_2 : Float(64)
      %learned_3 : Float(64)
      %learned_4 : Float(64)
      %learned_5 : Long()
      %learned_6 : Float(64, 64, 3, 3)
      %learned_7 : Float(64)
      %learned_8 : Float(64)
      %learned_9 : Float(64)
      %learned_10 : Float(64)
      %learned_11 : Long()
      %learned_12 : Float(64, 64, 3, 3)
      %learned_13 : Float(64)
      %learned_14 : Float(64)
      %learned_15 : Float(64)
      %learned_16 : Float(64)
      %learned_17 : Long()
      %learned_18 : Float(64, 64, 3, 3)
      %learned_19 : Float(64)
      %learned_20 : Float(64)
      %learned_21 : Float(64)
      %learned_22 : Float(64)
      %learned_23 : Long()
      %learned_24 : Float(64, 64, 3, 3)
      %learned_25 : Float(64)
      %learned_26 : Float(64)
      %learned_27 : Float(64)
      %learned_28 : Float(64)
      %learned_29 : Long()
      %learned_30 : Float(128,

/home/daniil/repos/pytorch_fresh/pytorch/build/lib.linux-x86_64-2.7/torch/nn/functional.py:1761: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [4]:
# restart here

convert_to_onnx.ipynb  resnet_18_8s_binary_512.onnx


In [3]:
import sys, os

# Run ```python setup.py build``` in the pytorch master repo
sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/build/lib.linux-x86_64-2.7")

# Run ```python setup.py build``` in the third_party/onnx folder
sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/third_party/onnx/build/lib.linux-x86_64-2.7/")

# Compile using cmake in the build folder
sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/build/")

import onnx

# Load the ONNX model
model = onnx.load("resnet_18_8s_binary_512.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %actual_input_1[FLOAT, 1x3x512x512]
) initializers (
  %learned_0[FLOAT, 64x3x7x7]
  %learned_1[FLOAT, 64]
  %learned_2[FLOAT, 64]
  %learned_3[FLOAT, 64]
  %learned_4[FLOAT, 64]
  %learned_5[INT64, scalar]
  %learned_6[FLOAT, 64x64x3x3]
  %learned_7[FLOAT, 64]
  %learned_8[FLOAT, 64]
  %learned_9[FLOAT, 64]
  %learned_10[FLOAT, 64]
  %learned_11[INT64, scalar]
  %learned_12[FLOAT, 64x64x3x3]
  %learned_13[FLOAT, 64]
  %learned_14[FLOAT, 64]
  %learned_15[FLOAT, 64]
  %learned_16[FLOAT, 64]
  %learned_17[INT64, scalar]
  %learned_18[FLOAT, 64x64x3x3]
  %learned_19[FLOAT, 64]
  %learned_20[FLOAT, 64]
  %learned_21[FLOAT, 64]
  %learned_22[FLOAT, 64]
  %learned_23[INT64, scalar]
  %learned_24[FLOAT, 64x64x3x3]
  %learned_25[FLOAT, 64]
  %learned_26[FLOAT, 64]
  %learned_27[FLOAT, 64]
  %learned_28[FLOAT, 64]
  %learned_29[INT64, scalar]
  %learned_30[FLOAT, 128x64x3x3]
  %learned_31[FLOAT, 128]
  %learned_32[FLOAT, 128]
  %learned_33[FLOAT, 128]
  %learned_34[F

In [4]:
# converting to a caffe2 native representation
# and saving to a protobuf file

from caffe2.python.onnx.backend import Caffe2Backend

init_net, predict_net = Caffe2Backend.onnx_graph_to_caffe2_net(model)

# Let's also save the init_net and predict_net to a file that we will later use for running them on mobile
with open('init_net.pb', "wb") as fopen:
    fopen.write(init_net.SerializeToString())
    
with open('predict_net.pb', "wb") as fopen:
    fopen.write(predict_net.SerializeToString())

In [1]:
# resort to a more low level interface -- without predictor but using our new fully converted weights
import sys, os

sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/build/lib.linux-x86_64-2.7")

# Adding onnx compiled from sources
sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/third_party/onnx/build/lib.linux-x86_64-2.7/")

# Adding caffe2 compiled from sources
sys.path.append("/home/daniil/repos/pytorch_fresh/pytorch/build/")



import numpy as np
import os, time
from caffe2.proto import caffe2_pb2
from caffe2.python import workspace

workspace.ResetWorkspace()

device_opts = caffe2_pb2.DeviceOption()
device_opts.device_type = caffe2_pb2.CUDA
device_opts.cuda_gpu_id = 1

init_def = caffe2_pb2.NetDef()
with open('init_net.pb', 'rb') as f:
    init_def.ParseFromString(f.read())
    init_def.device_option.CopyFrom(device_opts)
    workspace.RunNetOnce(init_def)

workspace.FeedBlob('actual_input_1', np.random.rand(1, 3, 512, 512).astype(np.float32), device_opts)
    
net_def = caffe2_pb2.NetDef()
with open('predict_net.pb', 'rb') as f:
    net_def.ParseFromString(f.read())
    net_def.device_option.CopyFrom(device_opts)
    workspace.CreateNet(net_def, overwrite=True)

#

In [2]:
def test():
    
    workspace.RunNet(net_def.name, 1)
    end = time.time()
    

In [6]:
%%timeit 
# 29.3 ms

test()

10 loops, best of 3: 29.2 ms per loop
